In [252]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
import re
import math
from collections import Counter
from urllib.parse import unquote
from typing import Union, Dict
from urllib.parse import urlparse, parse_qs

## Data Extraction

In [253]:
with open('part_1_sql_injection.json', 'r') as file:
    data = json.load(file)

In [254]:
if __name__ == "__main__":
    parsed_requests = []
    
    for entry in data:
        request = entry["request"]
        
        url = request.get("url", "")
        method = request.get("method", "")
        body = request.get("body", "")
        
        headers = request.get("headers", {})
        
        parsed_request = {
            "url": url,
            "method": method,
            "body": body,
            **headers
        }
        
        parsed_requests.append(parsed_request)
    
    df = pd.DataFrame(parsed_requests)

In [255]:
def extract_url_params(text: str) -> str:
    if '?' in text:
        text = text.split('?')
        text = unquote(text[1])

        return text
    return text

In [256]:
df.to_csv('with_sql_injection.csv', index=False)

## Data Pre-processing

In [257]:
read_df = pd.read_csv('with_sql_injection.csv')

read_df.loc[read_df['method'] == "POST", 'body'] = read_df.loc[read_df['method'] == "POST", 'body'].apply(unquote)
read_df.loc[read_df['method'] == "GET", 'url'] = read_df.loc[read_df['method'] == "GET", 'url'].apply(extract_url_params)

In [258]:
def calculate_entropy(text: str) -> float:
    if not isinstance(text, str) or not text.strip():
        return 0.0
    
    freq = Counter(text)
    total_length = len(text)

    entropy = sum(
        (-count / total_length) * math.log2(count / total_length) 
        for count in freq.values()
    )
    
    return entropy

In [259]:
def packet_length(packet_length: str) -> int:
    if not isinstance(packet_length, str) or not text.strip():
        return 0
    return int(packet_length)

In [260]:
sql_injection_patterns = [
    r"\bSELECT\b", r"\bINSERT\b", r"\bUPDATE\b", r"\bDELETE\b", r"\bDROP\b", r"\bUNION\b", r"\bJOIN\b",
    r"\bWHERE\b", r"\bORDER\s+BY\b", r"\bGROUP\s+BY\b", r"\bHAVING\b", r"\bEXEC\b", r"\bDECLARE\b",
    r"\bCASE\b", r"\bWHEN\b", r"\bTHEN\b", r"\bEND\b", r"\bIF\b", r"\bELSE\b", r"\bCAST\b", r"\bCONVERT\b",
    r"\bTABLE\b", r"\bCOLUMN\b", r"\bDATABASE\b", r"\bSCHEMA\b", r"\bINFORMATION_SCHEMA\b",
    r"\bTRUNCATE\b", r"\bALTER\b", r"\bCREATE\b", r"\bREPLACE\b", r"\bRENAME\b", r"\bGRANT\b", r"\bREVOKE\b",
    r"\bMERGE\b", r"\bINTERSECT\b", r"\bEXCEPT\b", r"\bEXECUTE\b", r"\bFETCH\b", r"\bOPEN\b", r"\bCLOSE\b",
    r"\bDEALLOCATE\b", r"\bUSE\b", r"\bLIMIT\b", r"\bOFFSET\b", r"\bNULL\b", r"\bISNULL\b", r"\bCOALESCE\b",
    r"\bXP_CMDSHELL\b", r"\bWAITFOR\s+DELAY\b", r"\bBENCHMARK\b", r"\bCHAR\b", r"\bASCII\b", r"\bHEX\b",
    r"\bCONCAT\b", r"\bSUBSTRING\b", r"\bMID\b", r"\bIFNULL\b", r"\bLOAD_FILE\b", r"\bOUTFILE\b",
    r"\bINTO\b", r"\bDUMPFILE\b", r"\bFLOOR\b", r"\bRAND\b", r"\bMD5\b", r"\bSHA1\b", r"\bPASSWORD\b",
    r"\bCURRENT_USER\b", r"\bSESSION_USER\b", r"\bSYSTEM_USER\b", r"\bUSER\b", r"\bVERSION\b",
    r"\bFOUND_ROWS\b", r"\bROW_COUNT\b", r"\bDATABASE\(\)\b", r"\bSCHEMA\(\)\b", r"\bTABLE_NAME\b",
    r"\bCOLUMN_NAME\b", r"\bCURRENT_TIMESTAMP\b", r"\bCURRENT_DATE\b", r"\bCURRENT_TIME\b",
    r"\bSESSION_ID\b", r"\bWAITFOR\s+TIME\b", r"\bEXEC\s+sp_executesql\b", r"\bEXEC\s+sp_sqlexec\b",
    r"\bSYSOBJECTS\b", r"\bSYSCOLUMNS\b", r"\bPG_SLEEP\b", r"\bEXTRACTVALUE\b", r"\bUPDATEXML\b",
    r"\bLTRIM\b", r"\bRTRIM\b", r"\bUPPER\b", r"\bLOWER\b", r"\bSYSADMIN\b", r"\bEXEC\s+MASTER\.DBO\.XP_CMDSHELL\b",
    r"\bINFORMATION_SCHEMA\.TABLES\b", r"\bINFORMATION_SCHEMA\.COLUMNS\b",
    r"--", r";", r"'", r"\"", r"/\*", r"\*/", r"\(", r"\)", r"\{", r"\}", r"\[", r"\]", r"<", r">",
    r"\|\|", r"\|", r"\^", r"\\", r"\@", r"\#", r"\%", r"\!", r"\$", r"\+", r"-", r"/", r"\*",
    r"\bOR\b", r"\bAND\b", r"\bLIKE\b", r"\bSLEEP\b", r"\bREGEXP\b", r"\bRLIKE\b",
    r"ORDER\s+BY\s+1--", r"UNION\s+SELECT", r"DROP\s+TABLE", r"ALTER\s+TABLE", r"INTO\s+OUTFILE",
    r"\b\d+\s*(=|!=|<|>|<=|>=)\s*\d+\b",
    r"\b\d+\s*[\+\-\*/%]\s*\d+\b",
    r"(\|\|)", r"\bCONCAT\s*\(", r"\bCHAR\s*\(", r"\bASCII\s*\(", r"\bUNHEX\s*\(",
    r"\b0x[0-9A-Fa-f]+\b",
    r"\b\d+\s*(&|\||\^)\s*\d+\b",
    r"--", r"#", r"/\*", r"\*/",
    r"\)\s*AND\s*\(",
    r"\bAND\b.*&.*\b",
    r"::", r"\@\@", r"\bAS\s+\w+",
    r"-\d+'?",
    r"^-?\d+'?",
]

sql_injection_regex = re.compile("|".join(sql_injection_patterns), re.IGNORECASE)

def count_sql_keywords(body: str) -> int:
    if not body:
        return 0

    matches = sql_injection_regex.findall(body)
    return len(matches)

In [261]:
def number_of_special_chars(request: str) -> int:
    if not isinstance(request, str) or not request.strip():
        return 0
    special_chars_pattern = r"[!@#$%^&*()_+\-=\[\]{};:'\"\\|,.<>?/`~]"
    special_chars = re.findall(special_chars_pattern, request)    
    return len(special_chars)

In [262]:
def url_length(request: Union[Dict, str]) -> int:
    if isinstance(request, str):
        return len(request)
    elif isinstance(request, dict):
        url = request.get('url', '')
        return len(url)
    else:
        return 0

In [263]:
def process_row(row):
    if row['method'] == 'GET':
        if pd.isna(row['url']):
            row['url'] = None
    else:
        if pd.isna(row['body']):
            row['body'] = None
    return row

In [264]:
def count_query_params(url: str) -> int:
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    return len(query_params)

In [265]:
def calculate_query_param_length(url: str) -> int:
    parsed_url = urlparse(url)
    query_string = parsed_url.query
    return len(query_string)

In [266]:
def calculate_path_depth(url: str) -> int:
    parsed_url = urlparse(url)
    path_segments = parsed_url.path.strip("/").split("/")
    return len(path_segments)

In [267]:
read_df['entropy'] = read_df.apply(
    lambda row: calculate_entropy(row['body']) if row['method'] == "POST" else calculate_entropy(row['url']), 
    axis=1
)
    
read_df['packet_length'] = read_df['Content-Length'].apply(packet_length)
read_df['sqli_keywords'] = read_df.apply(
    lambda row: count_sql_keywords(row['body']) if row['method'] == "POST" else count_sql_keywords(row['url']),
    axis=1
)
read_df['number_of_special_chars'] = read_df.apply(
    lambda row: number_of_special_chars(row['body']) if row['method'] == "POST" else number_of_special_chars(row['url']), axis=1
)
read_df['url_length'] = read_df['url'].apply(url_length)

read_df['query_param_count'] = read_df['url'].apply(count_query_params)
read_df['query_param_length'] = read_df['url'].apply(calculate_query_param_length)
read_df['path_depth'] = read_df['url'].apply(calculate_path_depth)

read_df['is_sql_injection'] = True
read_df = read_df.apply(process_row, axis=1)

In [268]:
read_df[['method','entropy','packet_length','sqli_keywords','number_of_special_chars','url_length','query_param_count','query_param_length','path_depth','is_sql_injection']].to_csv('with_sql_injection_dataset.csv',index=False)

In [269]:
read_df[['method','entropy','packet_length','sqli_keywords','number_of_special_chars','url_length','query_param_count','query_param_length','path_depth','is_sql_injection']]

,method,entropy,packet_length,sqli_keywords,number_of_special_chars,url_length,query_param_count,query_param_length,path_depth,is_sql_injection
0,POST,3.761446,0,7,13,39,0,0,1,True
1,POST,4.127569,0,5,8,39,0,0,1,True
2,POST,4.528841,0,9,11,39,0,0,1,True
3,POST,4.473839,0,9,11,39,0,0,1,True
4,POST,4.409742,0,7,9,39,0,0,1,True
...,...,...,...,...,...,...,...,...,...,...
434,POST,4.898913,0,30,22,39,0,0,1,True
435,POST,4.967038,0,29,23,39,0,0,1,True
436,POST,4.941078,0,29,23,39,0,0,1,True
437,POST,4.841228,0,29,23,39,0,0,1,True
